In [1]:
from py2neo import Graph,RelationshipMatcher,NodeMatcher
from json import dumps

In [2]:
graph = Graph("bolt://127.0.0.1:7687",username='neo4j',password='password')

In [3]:
re_matcher = RelationshipMatcher(graph)
no_matcher = NodeMatcher(graph)

In [4]:
def recursive_search(cls):
    # gather object
    label = ':Object'
    name = cls
    object_node = graph.run("MATCH (n:Object {name:\"%s\"}) return n"%(cls))
    if not object_node.forward():
        object_node = graph.run("MATCH (n:Shape {name:\"%s\"}) return n"%(cls))
        label = ':Shape'
        if not object_node.forward():
            return {}
    object_node = dict(object_node.current.values()[0])
    
    node_list = {}
    node_list[label] = set()
    node_list[label].add(name)
    search_list = []
    search_list.append((label,name))
    
    root_nodes = []
    
    # Search entire digraph with object node as begin node
    while len(search_list) > 0:
        all_nodes = graph.run("MATCH (n%s {name:\"%s\"})-[]->(m) return m"%(search_list[0][0],search_list[0][1])).data()
        for node in all_nodes:
            label = str(node['m'].labels)
            name = dict(node['m'])['name']
            if label not in node_list:
                node_list[label] = set()
            if name not in node_list[label]:
                node_list[label].add(name)
                search_list.append((label,name))
        if len(all_nodes)==0:
            root_nodes.append(search_list[0])
        search_list = search_list[1:]
    
    # Remove root node for each type of nodes
    for i in root_nodes:
        node_list[i[0]].remove(i[1])
        
    # Remove countered grasp types
    if ':Concept' in node_list:
        countered_grasp_method = []
        for grasp_type in node_list[':GraspMethod']:
            counter_concepts = graph.run("MATCH (m)-[:CantApply]->(n:GraspMethod {name:\"%s\"}) return m"%(grasp_type)).data()
            for counter_concept in counter_concepts:
                if dict(counter_concept['m'])['name'] in node_list[':Concept']:
                    countered_grasp_method.append(grasp_type)
                    break
        for grasp_type in countered_grasp_method:
            node_list[':GraspMethod'].remove(grasp_type)
            
    # convert results to lists
    for i in node_list:
        node_list[i] = list(node_list[i])
        
    return node_list

In [8]:
print(dumps(recursive_search("Mark Cup"),indent='  '))

{
  ":Object": [
    "Cup",
    "Wall of Cup",
    "Container",
    "Drink Container",
    "Mark Cup",
    "Cup Handle"
  ],
  ":Shape": [
    "Flared Handle",
    "Cylinder"
  ],
  ":GraspMethod": [
    "Attract Grasping",
    "Hold Grasping",
    "Pinch Grasping"
  ]
}


In [193]:
# def direct_search(cls = "Mark Cup"):
#     # gather object
#     object_node = dict(graph.run("MATCH (n:Object {name:\"%s\"}) return n"%(cls)).next().values()[0])
#     all_nodes = graph.run("MATCH (n:Object {name:\"%s\"})-[r:Assembled_Of]->(m) return m"%(cls))
    
#     # gather subparts, if None, add object's self as a subpart
#     subparts = []
#     while all_nodes.forward():
#         subparts.append(dict(all_nodes.current.values()[0]))
#     if len(subparts)==0:
#         subparts.append(object_node)
#     subparts = {i['name']:i['description'] for i in subparts}
    
#     # gather shapes
#     for part_name in subparts:
#         description = subparts[part_name]
#         subparts[part_name] = {"description":description,"shapes":[]}
#         formed_shapes = graph.run("MATCH (n:Object {name:\"%s\"})-[r:Shape_Of]->(m) return m"%(part_name))
#         shapes = []
#         while formed_shapes.forward():
#             shapes.append(dict(formed_shapes.current.values()[0]))
#         subparts[part_name]["shapes"] = {i['name']:{"description":i['description'],"grasps":[]} for i in shapes}
    
#     # gather grasp types
#     all_available_grasps = {}
#     for part_name in subparts:
#         for shape_name in subparts[part_name]["shapes"]:
#             grasp_choices = graph.run("MATCH (n:Shape {name:\"%s\"})-[r:CanApply]->(m) return m"%(shape_name))
#             grasps = []
#             while grasp_choices.forward():
#                 grasps.append(dict(grasp_choices.current.values()[0]))
#             subparts[part_name]["shapes"][shape_name]['grasps'] = {i['name']:{"description":i['description']} for i in grasps}
#             for grasp_type in subparts[part_name]["shapes"][shape_name]['grasps']:
#                 if grasp_type not in all_available_grasps:
#                     all_available_grasps[grasp_type] = 0
#                 all_available_grasps[grasp_type] += 1
#     return subparts,all_available_grasps

In [194]:
# gathered_info, gathered_grasp_types = direct_search("Cup")
# print("grasp*Shape_cnts: ",dumps(gathered_grasp_types,indent='  '))
# print()
# print("tree display of info gathered: ",dumps(gathered_info,indent='  '))

In [ ]:
知识库的更新